In [1]:
import numpy as np
# Loading the pose training and validating data
x_train = np.load("maleTrain/male_training_data.npy", allow_pickle=True)
y_train = np.load("maleTrain/male_training_data_labels.npy", allow_pickle=True)
x_validate = np.load("maleTrain/female_training_data.npy", allow_pickle=True)
y_validate = np.load("maleTrain/female_training_data_labels.npy", allow_pickle=True)
x_train=np.asarray(x_train).astype(np.float32)
y_train=np.asarray(y_train).astype(np.float32)
x_validate = np.asarray(x_validate).astype(np.float32)
y_validate = np.asarray(y_validate).astype(np.float32)

print(x_train.shape)
print(y_train.shape)
print(x_validate.shape)
print(y_validate.shape)

(220, 60, 25, 2)
(220, 60, 4)
(148, 60, 25, 2)
(148, 60, 4)


In [5]:
# CNN + LSTM model
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers.pooling import GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, LSTM
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam
from keras.layers import Conv2D
from keras.layers import MaxPool1D, MaxPool2D
import tensorflow as tf

cnn = Sequential()
cnn.add(TimeDistributed(keras.layers.Conv1D(filters=64, kernel_size=3,padding='same', activation='relu'), input_shape=x_train.shape[1:]))
cnn.add(TimeDistributed(BatchNormalization()))
cnn.add(TimeDistributed(MaxPooling1D(2,2)))
cnn.add(TimeDistributed(keras.layers.Conv1D(filters=64, kernel_size=3,padding='same', activation='relu'), input_shape=x_train.shape[1:]))
cnn.add(TimeDistributed(BatchNormalization()))
cnn.add(TimeDistributed(MaxPooling1D(2,2)))
cnn.add(TimeDistributed(Dropout(0.5)))
cnn.add(BatchNormalization())
cnn.add(TimeDistributed(Flatten()))
cnn.add(LSTM(40,unit_forget_bias=0.4, return_sequences=True))
cnn.add(TimeDistributed(Dense(4, activation='softmax')))
cnn.summary()
print(cnn.output_shape)



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_9 (TimeDist (None, 60, 25, 64)        448       
_________________________________________________________________
time_distributed_10 (TimeDis (None, 60, 25, 64)        256       
_________________________________________________________________
time_distributed_11 (TimeDis (None, 60, 12, 64)        0         
_________________________________________________________________
time_distributed_12 (TimeDis (None, 60, 12, 64)        12352     
_________________________________________________________________
time_distributed_13 (TimeDis (None, 60, 12, 64)        256       
_________________________________________________________________
time_distributed_14 (TimeDis (None, 60, 6, 64)         0         
_________________________________________________________________
time_distributed_15 (TimeDis (None, 60, 6, 64)        

In [6]:
# Compiling the model
adam = Adam(lr=0.0001)

cnn.compile(loss='categorical_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])

In [ ]:
# history = cnn.fit(x_train, y_train, validation_data=(x_validate, y_validate), epochs=200)

In [8]:
# Load the weights
cnn.load_weights("weights/maleTrained.h5")

In [9]:
# # validating the model on training and validation data
_, train_acc = cnn.evaluate(x_train, y_train)
_, val_acc = cnn.evaluate(x_validate,y_validate)

5/5 [==============================] - 0s 19ms/step - loss: 0.4963 - accuracy: 0.8662


In [10]:
import statistics

In [11]:
motion = ["knocking", "lifting", "throwing", "walking"]

In [15]:
x_test = np.load("data/ale_knock.npy", allow_pickle=True)
x_test = x_test[:240,...]
x_test = np.asarray(x_test).astype(np.float32)
x_test = x_test.reshape((4,60,25,2))

In [16]:
Yht_pred = cnn.predict(np.asarray(x_test))
a = np.argmax(Yht_pred[0],axis = 1)
motion[statistics.mode(np.argmax(Yht_pred[1],axis = 1))]

'knocking'